In [24]:
def correlation_score(y_true, y_pred):
    corrsum = 0
    for i in range(y_true.shape[0]):
        corrsum += np.corrcoef(y_true[i], y_pred[i])[1, 0]
    return corrsum / len(y_true)

In [2]:
LR_START = 0.01
BATCH_SIZE = 512

def create_model():
    
    reg1 = 9.613e-06
    reg2 = 1e-07
    REG1 = tf.keras.regularizers.l2(reg1)
    REG2 = tf.keras.regularizers.l2(reg2)
    DROP = 0.1

    activation = 'selu'
    inputs = Input(shape =(X.shape[1],))

    x0 = Dense(512, 
              kernel_regularizer = REG1,
              activation = activation,
             )(inputs)
    x0 = Dropout(DROP)(x0)
    
    
    x1 = Dense(1024, 
               kernel_regularizer = REG1,
               activation = activation,
             )(x0)
    x1 = Dropout(DROP)(x1)
    
    
    x2 = Dense(1024, 
               kernel_regularizer = REG1,
               activation = activation,
             )(x1) 
    x2= Dropout(DROP)(x2)
    
    x3 = Dense(Y.shape[1],
               kernel_regularizer = REG1,
               activation = activation,
             )(x2)
    x3 = Dropout(DROP)(x3)

         
    x = Concatenate()([
                x0, 
                x1, 
                x2, 
                x3
                ])
    
    x = Dense(Y.shape[1], 
                kernel_regularizer = REG2,
                activation='linear',
                )(x)
    
    
    model = Model(inputs, x)
    

    return model

In [10]:
X = scipy.sparse.load_npz('../sparse/train_cite_inputs_values.sparse.npz')
X[X != 0] = 1

In [13]:
svd_x = TruncatedSVD(n_components = 128)
X = svd_x.fit_transform(X)

In [14]:
X_copy = X

In [25]:
targets = scipy.sparse.load_npz("../sparse/train_multi_targets_values.sparse.npz")
targets = targets
targets = scprep.filter.remove_empty_genes(targets)
# Normalize the targets row-wise: This doesn't change the correlations,
# and negative_correlation_loss depends on it
targets -= targets.mean(axis=1).reshape(-1, 1)
targets /= targets.std(axis=1).reshape(-1, 1)
targets = np.array(targets)
svd = TruncatedSVD(n_components = 512,random_state = 1)
Y = svd.fit_transform(targets)
with np.load('../sparse/train_multi_inputs_idxcol.npz',allow_pickle=True) as data:
    cell_index = data['index']
with np.load('../sparse/test_multi_inputs_idxcol.npz',allow_pickle=True) as data:
    cell_index_t = data['index']
metadata_df = pd.read_csv('../../input/open-problems-multimodal/metadata.csv', index_col='cell_id')
metadata_df = metadata_df[metadata_df.technology=="multiome"]
meta = metadata_df.reindex(cell_index)

KeyboardInterrupt: 

In [8]:
inputs_both_128 = np.load("../multi_pca/inputs_both_128.npy")
X = inputs_both_128[:targets_copy.shape[0],:40]

In [16]:
targets_copy = targets

In [10]:
meta['range'] = range(len(meta))
day7_donor1 = meta.loc[(meta.day == 7)&(meta.donor == 32606)].range.values
day4_donor1 = meta.loc[(meta.day == 4)&(meta.donor == 32606)].range.values
X_tr = X[day4_donor1][:1000,:40]
X_va = X[day7_donor1][:1000,:40]
svd = TruncatedSVD(n_components = 512,random_state = 1)
svd.fit(targets_copy[day7_donor1])
y_tr = svd.transform(targets_copy[day4_donor1][:1000])[:,:64]
y_va = svd.transform(targets_copy[day7_donor1][:1000])[:,:64]
targets = targets_copy[day7_donor1][:1000]
print(X_tr.shape)
print(y_va.shape)
print(targets.shape)
Y = y_tr
X = X_tr

(1000, 40)
(1000, 64)
(1000, 23418)


In [14]:
(targets).shape

(1000, 23418)

In [25]:
y_va_pred = model.predict(X_va)
correlation_score(targets.toarray(),y_va_pred@svd.components_[:64])

32/32 [==============================] - 0s 2ms/step


0.6115472329332223

In [11]:
import warnings
warnings.filterwarnings("ignore")

VERBOSE = 0
N_SPLIT = 3
#kf = KFold(n_splits=N_SPLIT, shuffle=True, random_state=42)
kf = KFold(n_splits = N_SPLIT)

scores = []
pca_scores = []
pred_train = np.zeros_like(Y)

model = create_model()

lr = ReduceLROnPlateau(
            monitor = "val_loss",
            factor = 0.9, 
            patience = 4, 
            verbose = VERBOSE)

es = EarlyStopping(
            monitor = "val_loss",
            patience = 30, 
            verbose = VERBOSE,
            mode = "min", 
            restore_best_weights = True)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
              loss = 'mse',
              metrics=None)
model.fit(X_tr,
          y_tr,
          validation_data=(X_va,y_va),
          epochs =500,
          verbose = VERBOSE,
          batch_size=256,
          callbacks = [es,lr]
         )
y_va_pred = model.predict(X_va)
corrscore = correlation_score(targets, y_va_pred@svd.components_[:64])
scores.append(corrscore)
pca_corrscore = correlation_score(y_va[:,:64], y_va_pred[:,:64])
pca_scores.append(pca_corrscore)

print(f'Mean squared error = {np.round(mean_squared_error(y_va,y_va_pred),2)}')
print(f'Correlation = {corrscore:.5f}')
print(f'PCA Correlation = {pca_corrscore:.5f}')

del X_tr,X_va,y_tr,y_va
gc.collect()

# Show overall score
print(f"{Fore.GREEN}{Style.BRIGHT}Mean corr = {np.array(scores).mean():.5f}{Style.RESET_ALL}")
print(f"{Fore.RED}{Style.BRIGHT}Mean pca corr = {np.array(pca_scores).mean():.5f}{Style.RESET_ALL}")
# score_total = correlation_score(targets, pred_train@svd.components_)#pred_train@pca_target_comps)
# print(f"{Fore.BLUE}{Style.BRIGHT}Oof corr   = {score_total:.5f}{Style.RESET_ALL}")

32/32 [==============================] - 0s 2ms/step


ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 1 and the array at index 1 has size 23418

In [12]:
with np.load('../sparse/train_cite_inputs_idxcol.npz',allow_pickle=True) as data:
    cell_columns_cite = data['columns']
cell_columns_cite = [i.split('_')[0] for i in cell_columns_cite]
with np.load('../sparse/train_multi_targets_idxcol.npz',allow_pickle=True) as data:
    cell_columns_multi = data['columns']
mask_cite = [i for i,x in enumerate(cell_columns_cite) if x in cell_columns_multi]
mask_multi = [i for i,x in enumerate(cell_columns_multi) if x in cell_columns_cite]

In [44]:
len(mask_cite)

18386

In [41]:
len(mask_cite)

18386

In [42]:
len(mask_multi)

18386

In [4]:
X = scipy.sparse.load_npz('../sparse/train_cite_inputs_values.sparse.npz')
X = X[:,mask_cite]
X.shape

(70988, 18386)

In [5]:
Y = pd.read_hdf("../../input/open-problems-multimodal/train_cite_targets.h5")
#Y = Y.loc[meta.day == 3]
Y = Y.values
Y -= Y.mean(axis=1).reshape(-1, 1)
Y /= Y.std(axis=1).reshape(-1, 1)
Y.shape

(70988, 140)

In [13]:
Y = Y[:700]
X = X[:700]

In [14]:
Y.shape

(700, 140)

In [18]:
LR_START = 0.01
BATCH_SIZE = 512

def create_model(input_shape, output_shape):
    
    reg1 = 9.613e-06
    reg2 = 1e-07
    REG1 = tf.keras.regularizers.l2(reg1)
    REG2 = tf.keras.regularizers.l2(reg2)
    DROP = 0.1

    activation = 'selu'
    inputs = Input(shape =(input_shape,))

    x0 = Dense(256, 
              kernel_regularizer = REG1,
              activation = activation,
             )(inputs)
    x0 = Dropout(DROP)(x0)
    
    
    x1 = Dense(512, 
               kernel_regularizer = REG1,
               activation = activation,
             )(x0)
    x1 = Dropout(DROP)(x1)
    
    
    x2 = Dense(512, 
               kernel_regularizer = REG1,
               activation = activation,
             )(x1) 
    x2= Dropout(DROP)(x2)
    
    x3 = Dense(output_shape,
               kernel_regularizer = REG1,
               activation = activation,
             )(x2)
    x3 = Dropout(DROP)(x3)

         
    x = Concatenate()([
                x0, 
                x1, 
                x2, 
                x3
                ])
    
    x = Dense(output_shape, 
                kernel_regularizer = REG2,
                activation='linear',
                )(x)
    
    
    model = Model(inputs, x)
    

    return model

In [2]:
%ls ../multi_model_saves

model_2_13176_0/        model_3_13176_2/  model_4_32606_0/
model_2_13176_1/        model_3_31800_0/  model_4_32606_1/
model_2_13176_2/        model_3_31800_1/  model_4_32606_2/
model_2_13176_svds.npy  model_3_31800_2/  model_7_13176_0/
model_2_31800_0/        model_3_32606_0/  model_7_13176_1/
model_2_31800_1/        model_3_32606_1/  model_7_13176_2/
model_2_31800_2/        model_3_32606_2/  model_7_31800_0/
model_2_32606_0/        model_4_13176_0/  model_7_31800_1/
model_2_32606_1/        model_4_13176_1/  model_7_31800_2/
model_2_32606_2/        model_4_13176_2/  model_7_32606_0/
model_2_32606_svds.npy  model_4_31800_0/  model_7_32606_1/
model_3_13176_0/        model_4_31800_1/  model_7_32606_2/
model_3_13176_1/        model_4_31800_2/


In [22]:
# Overall baseline

N_SPLITS = 3

X = scipy.sparse.load_npz('../sparse/train_cite_inputs_values.sparse.npz')
X = X[:7000]
X = X[:,mask_cite]
svd_cite = TruncatedSVD(n_components = 128)
X = svd_cite.fit_transform(X)
X = X[:,:64]

Y = pd.read_hdf("../../input/open-problems-multimodal/train_cite_targets.h5")
Y = Y.values
Y -= Y.mean(axis=1).reshape(-1, 1)
Y /= Y.std(axis=1).reshape(-1, 1)

Y = Y[:7000]

import warnings
warnings.filterwarnings("ignore")

EPOCHS = 300 

pred_train = np.zeros((Y.shape[0],Y.shape[1]))

np.random.seed(1)
tf.random.set_seed(1)
score_list = []
kf = KFold(n_splits=N_SPLITS)

for fold, (idx_tr, idx_va) in enumerate(kf.split(X)):
    start_time = datetime.datetime.now()
    model_cite = None
    gc.collect()
    
    X_tr = X[idx_tr]
    y_tr = Y[idx_tr]
    X_va = X[idx_va]
    y_va = Y[idx_va]

    lr = ReduceLROnPlateau(
                    monitor = "val_loss",
                    factor = 0.9, 
                    patience = 4, 
                    verbose = VERBOSE)

    es = EarlyStopping(
                    monitor = "val_loss",
                    patience = 40, 
                    verbose = VERBOSE,
                    mode = "min", 
                    restore_best_weights = True)

    model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
                    filepath = './citeseq',
                    save_weights_only = True,
                    monitor = 'val_loss',
                    mode = 'min',
                    save_best_only = True)

    callbacks = [
                    lr, 
                    es, 
                    model_checkpoint_callback
                    ]
    
    model_cite = create_model(X.shape[1],Y.shape[1])
    
    model_cite.compile(
                optimizer = tf.keras.optimizers.Adam(learning_rate=LR_START),
                metrics = [negative_correlation_loss],
                loss = negative_correlation_loss
                 )
    # Training
    model_cite.fit(
                X_tr,
                y_tr, 
                validation_data=(
                                X_va,
                                y_va), 
                epochs = EPOCHS,
                verbose = VERBOSE,
                batch_size = BATCH_SIZE,
                shuffle = True,
                callbacks = callbacks)

    del X_tr, y_tr 
    gc.collect()
    
    model_cite.save(f"./submissions/model_donor1_day1")
    print('model saved')
    
    #  Model validation
    y_va_pred = model_cite.predict(X_va)
    corrscore = correlation_score(y_va, y_va_pred)
    pred_train[idx_va] = y_va_pred
    
    print(f"Fold {fold}, correlation =  {corrscore:.5f}")
    del X_va, y_va, y_va_pred
    gc.collect()
    score_list.append(corrscore)
    break

model saved
73/73 [==============================] - 0s 2ms/step
Fold 0, correlation =  0.89362


In [4]:
model_cite = load_model(f"./submissions/model_donor1_day1",
                        custom_objects={'negative_correlation_loss': negative_correlation_loss})

2022-11-07 18:12:17.492561: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-07 18:12:17.499318: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2022-11-07 18:12:17.499338: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1934] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2022-11-07 18:12:17.499701: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow bin

In [5]:
targets_copy = scipy.sparse.load_npz("../sparse/train_multi_targets_values.sparse.npz")

In [3]:
with np.load('../sparse/train_multi_inputs_idxcol.npz',allow_pickle=True) as data:
    cell_index = data['index']
with np.load('../sparse/test_multi_inputs_idxcol.npz',allow_pickle=True) as data:
    cell_index_t = data['index']
metadata_df = pd.read_csv('../../input/open-problems-multimodal/metadata.csv', index_col='cell_id')
metadata_df = metadata_df[metadata_df.technology=="multiome"]
meta = metadata_df.reindex(cell_index)
meta['range'] = range(len(meta))
day7_donor1 = meta.loc[(meta.day == 7)&(meta.donor == 32606)].range.values

In [62]:
targets_copy = targets_copy[:,mask_multi]
targets_copy = scprep.filter.remove_empty_genes(targets_copy)

In [63]:
targets = targets_copy.copy()
targets = targets_copy[day7_donor1]
# Normalize the targets row-wise: This doesn't change the correlations,
# and negative_correlation_loss depends on it
targets -= targets.mean(axis=1).reshape(-1, 1)
targets /= targets.std(axis=1).reshape(-1, 1)
targets = np.array(targets)
svd = TruncatedSVD(n_components = 512,random_state = 1)
svd.fit(targets)
Y = svd.transform(targets)

In [31]:
inputs_both_128 = np.load("../multi_pca/inputs_both_128.npy")
X = inputs_both_128[:targets_copy.shape[0],:40]

In [32]:
X_copy = X.copy()

In [33]:
X = X_copy[day7_donor1][:1000,:40]
Y = svd.transform(targets[:1000])[:,:64]
targets = targets[:1000]
print(X.shape)
print(Y.shape)
print(targets.shape)

(1000, 40)
(1000, 64)
(1000, 23418)


In [75]:
import warnings
warnings.filterwarnings("ignore")

VERBOSE = 0
N_SPLIT = 3
#kf = KFold(n_splits=N_SPLIT, shuffle=True, random_state=42)
kf = KFold(n_splits = N_SPLIT)

scores = []
pca_scores = []
Y_pred = np.zeros_like(Y)

for fold,(idx_tr, idx_va) in enumerate(kf.split(X)):
    
    X_tr = X[idx_tr]
    y_tr = Y[idx_tr]
    
    X_va = X[idx_va]
    y_va = Y[idx_va] 

    model = create_model()

    lr = ReduceLROnPlateau(
                monitor = "val_loss",
                factor = 0.9, 
                patience = 4, 
                verbose = VERBOSE)

    es = EarlyStopping(
                monitor = "val_loss",
                patience = 30, 
                verbose = VERBOSE,
                mode = "min", 
                restore_best_weights = True)

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
                  loss = 'mse',
                  metrics=None)
    model.fit(X_tr,
              y_tr,
              validation_data=(X_va,y_va),
              epochs =500,
              verbose = VERBOSE,
              batch_size=256,
              callbacks = [es,lr]
             )
    y_va_pred = model.predict(X_va)
    Y_pred[idx_va] = y_va_pred
    corrscore = correlation_score(targets[idx_va], y_va_pred@svd.components_[:64])
    scores.append(corrscore)
    pca_corrscore = correlation_score(y_va[:,:64], y_va_pred[:,:64])
    pca_scores.append(pca_corrscore)

    print(f'\n --------- FOLD {fold} -----------')
    print(f'Mean squared error = {np.round(mean_squared_error(y_va,y_va_pred),2)}')
    print(f'Correlation = {corrscore:.5f}')
    print(f'PCA Correlation = {pca_corrscore:.5f}')

    filename = f"model_{fold}"
    model.save(filename)
    print('model saved :',filename)

    del X_tr,X_va,y_tr,y_va
    gc.collect()

# Show overall score
print(f"{Fore.GREEN}{Style.BRIGHT}Mean corr = {np.array(scores).mean():.5f}{Style.RESET_ALL}")
print(f"{Fore.RED}{Style.BRIGHT}Mean pca corr = {np.array(pca_scores).mean():.5f}{Style.RESET_ALL}")
# score_total = correlation_score(targets, pred_train@svd.components_)#pred_train@pca_target_comps)
# print(f"{Fore.BLUE}{Style.BRIGHT}Oof corr   = {score_total:.5f}{Style.RESET_ALL}")

11/11 [==============================] - 0s 2ms/step

 --------- FOLD 0 -----------
Mean squared error = 7.489999771118164
Correlation = 0.61171
PCA Correlation = 0.96735
model saved : model_0
11/11 [==============================] - 0s 2ms/step

 --------- FOLD 1 -----------
Mean squared error = 7.079999923706055
Correlation = 0.61946
PCA Correlation = 0.96936
model saved : model_1
11/11 [==============================] - 0s 2ms/step

 --------- FOLD 2 -----------
Mean squared error = 7.239999771118164
Correlation = 0.61784
PCA Correlation = 0.96855
model saved : model_2
Mean corr = 0.61634
Mean pca corr = 0.96842


In [76]:
model_cite = load_model(f"./submissions/model_donor1_day1",
                        custom_objects={'negative_correlation_loss': negative_correlation_loss})
prot = model_cite.predict(Y_pred)

32/32 [==============================] - 0s 2ms/step


In [77]:
targets_copy = scipy.sparse.load_npz("../sparse/train_multi_targets_values.sparse.npz")
targets_copy = scprep.filter.remove_empty_genes(targets_copy)
targets = targets_copy.copy()
targets = targets_copy[day7_donor1]
# Normalize the targets row-wise: This doesn't change the correlations,
# and negative_correlation_loss depends on it
targets -= targets.mean(axis=1).reshape(-1, 1)
targets /= targets.std(axis=1).reshape(-1, 1)
targets = np.array(targets)
svd = TruncatedSVD(n_components = 512,random_state = 1)
svd.fit(targets)
Y = svd.transform(targets)

In [79]:
inputs_both_128 = np.load("../multi_pca/inputs_both_128.npy")
X = inputs_both_128[:targets_copy.shape[0],:40]
X = X[day7_donor1][:1000,:40]
Y = svd.transform(targets[:1000])[:,:64]
targets = targets[:1000]
X = np.hstack([X,prot])
print(X.shape)
print(Y.shape)
print(targets.shape)

(1000, 180)
(1000, 64)
(1000, 22858)


In [80]:
import warnings
warnings.filterwarnings("ignore")

VERBOSE = 0
N_SPLIT = 3
#kf = KFold(n_splits=N_SPLIT, shuffle=True, random_state=42)
kf = KFold(n_splits = N_SPLIT)

scores = []
pca_scores = []
Y_pred = np.zeros_like(Y)

for fold,(idx_tr, idx_va) in enumerate(kf.split(X)):
    
    X_tr = X[idx_tr]
    y_tr = Y[idx_tr]
    
    X_va = X[idx_va]
    y_va = Y[idx_va] 

    model = create_model()

    lr = ReduceLROnPlateau(
                monitor = "val_loss",
                factor = 0.9, 
                patience = 4, 
                verbose = VERBOSE)

    es = EarlyStopping(
                monitor = "val_loss",
                patience = 30, 
                verbose = VERBOSE,
                mode = "min", 
                restore_best_weights = True)

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
                  loss = 'mse',
                  metrics=None)
    model.fit(X_tr,
              y_tr,
              validation_data=(X_va,y_va),
              epochs =500,
              verbose = VERBOSE,
              batch_size=256,
              callbacks = [es,lr]
             )
    y_va_pred = model.predict(X_va)
    Y_pred[idx_va] = y_va_pred
    corrscore = correlation_score(targets[idx_va], y_va_pred@svd.components_[:64])
    scores.append(corrscore)
    pca_corrscore = correlation_score(y_va[:,:64], y_va_pred[:,:64])
    pca_scores.append(pca_corrscore)

    print(f'\n --------- FOLD {fold} -----------')
    print(f'Mean squared error = {np.round(mean_squared_error(y_va,y_va_pred),2)}')
    print(f'Correlation = {corrscore:.5f}')
    print(f'PCA Correlation = {pca_corrscore:.5f}')

    filename = f"model_{fold}"
    model.save(filename)
    print('model saved :',filename)

    del X_tr,X_va,y_tr,y_va
    gc.collect()

# Show overall score
print(f"{Fore.GREEN}{Style.BRIGHT}Mean corr = {np.array(scores).mean():.5f}{Style.RESET_ALL}")
print(f"{Fore.RED}{Style.BRIGHT}Mean pca corr = {np.array(pca_scores).mean():.5f}{Style.RESET_ALL}")
# score_total = correlation_score(targets, pred_train@svd.components_)#pred_train@pca_target_comps)
# print(f"{Fore.BLUE}{Style.BRIGHT}Oof corr   = {score_total:.5f}{Style.RESET_ALL}")

11/11 [==============================] - 0s 2ms/step

 --------- FOLD 0 -----------
Mean squared error = 10.470000267028809
Correlation = 0.61938
PCA Correlation = 0.96431
model saved : model_0
11/11 [==============================] - 0s 2ms/step

 --------- FOLD 1 -----------
Mean squared error = 10.239999771118164
Correlation = 0.62692
PCA Correlation = 0.96559
model saved : model_1
11/11 [==============================] - 0s 2ms/step

 --------- FOLD 2 -----------
Mean squared error = 10.640000343322754
Correlation = 0.62486
PCA Correlation = 0.96431
model saved : model_2
Mean corr = 0.62372
Mean pca corr = 0.96474


In [ ]:
day4_donor1 = meta.loc[(meta.day == 4)&(meta.donor == 32606)].range.values
targets_copy = scipy.sparse.load_npz("../sparse/train_multi_targets_values.sparse.npz")
targets_copy = scprep.filter.remove_empty_genes(targets_copy)
targets = targets_copy.copy()
targets = targets_copy[day7_donor1]
# Normalize the targets row-wise: This doesn't change the correlations,
# and negative_correlation_loss depends on it
targets -= targets.mean(axis=1).reshape(-1, 1)
targets /= targets.std(axis=1).reshape(-1, 1)
targets = np.array(targets)
svd = TruncatedSVD(n_components = 512,random_state = 1)
svd.fit(targets)
Y = svd.transform(targets_copy[day4_donor1])

In [26]:
model_cite.predict(

In [28]:
X.shape

(1000, 40)

In [29]:
X = scipy.sparse.load_npz('../sparse/train_cite_inputs_values.sparse.npz')
X = X[:7000]
X = X[:,mask_cite]
svd_cite = TruncatedSVD(n_components = 128)
X = svd_cite.fit_transform(X)
X = X[:,:64]

KeyboardInterrupt: 